## Бизнес-процесс
Клиент оформляет заявку на сайте банка на какой-либо продукт. \
Сайт отправляет заявку в CRM-систему, которая присваивает свой внутренний id заявке и запускает процесс кредитного скоринга \
и выдачи продукта в случае положительного решения для клиента.

### Описание данных
В приложении к заданию дан набор данных в файле **dataset.xlsx**. \
Листы SITE и CRM являются таблицами в базе данных. Таблица SITE содержит заявки, сформированные на сайте. \
Таблица CRM содержит id заявки в CRM-системе, id заявки в системе сайта, id клиента, продукт, \
на который оформлена заявка, решение (одобрено / нет) и наличие факта выдачи продукта. 

### Задания
#### Задание 1.
Напишите SQL-запрос, который бы возвращал таблицу, на основе которой можно построить воронку продаж от количества заявок на сайте до выдачи с группировкой по продукту.\
Выдвиньте гипотезу о причине расхождений в количестве заявок между системой сайта и crm-системой.

#### Задание 2.
Клиенты могут совершать повторные заявки. Характеристики клиентов, которые делают повторные заявки и которые их не делают, отличаются. Необходимо отсегментировать заявки из таблицы CRM исходя из того, сколько всего клиент сделал заявок. Напишите SQL-запрос, который бы рассчитывал дополнительное поле для таблицы CRM, где находился бы сегмент клиента по признаку количества сделанных заявок (шаг для сегментации задайте произвольно).
Рассмотрите воронку продаж в разрезе полученных сегментов.
Какие закономерности это позволило выявить? Предложите объяснение.

#### Задание 3.
Напишите sql-запрос, который бы исходя из данных таблицы CRM рассчитывал бы таблицу, содержащую три поля, где id клиента являлось бы уникальным значением, а также признаки «клиент делал заявку на кредит» и «клиент делал заявку на кредитную карту» (1 – делал, неважно сколько раз, 0 – не делал).

#### Задание 4.
Напишите SQL-запрос, который исходя из данных таблицы CRM возвращал бы предыдущую заявку клиента на этот же продукт для текущей заявки (например, если текущая заявка – на кредитную карту, то в поле должно содержаться id предыдущей заявки на кредитную карту, а не на кредит).
Результат должен содержать поля: APPLICATION_ID, CLIENT_ID, ASK_PROD_TYPE_NM, INTEGRATION_ID, PREV_APPLICATION_ID (предыдущая заявка)


In [1]:
import sqlite3
import pandas as pd

In [2]:
# прочитать с помощью Pandas данные из excel
# YOUR CODE HERE

data_site = pd.read_excel(r'C:\Users\DAN\Desktop\work\TEST SQL_1.xlsx', sheet_name='SITE')

data_crm = pd.read_excel(r'C:\Users\DAN\Desktop\work\TEST SQL_1.xlsx', sheet_name='CRM')

In [3]:
data_site.head()

,APPLICATION_ID
0,SITE-10
1,SITE-1
2,SITE-4
3,SITE-22
4,SITE-11


In [4]:
data_crm.head()

,APPLICATION_ID,CLIENT_ID,INTEGRATION_ID,ASK_PROD_TYPE_NM,APPROVED_FLG,ISSUED_FLG
0,1,52363,SITE-10,Кредит,1,0
1,2,69440,SITE-1,Кредитная карта,0,0
2,3,69387,SITE-4,Кредитная карта,0,0
3,4,36849,SITE-22,Кредит,1,0
4,5,69487,SITE-11,Кредитная карта,0,0


In [5]:
#проверка на дубли в APPLICATION_ID, INTEGRATION_ID
# YOUR CODE HERE

print('Количество дублей в таблице data_site в колонке APPLICATION_ID составляет:',len(data_site['APPLICATION_ID'])- len(data_site['APPLICATION_ID']. drop_duplicates()))
print('Количество дублей в таблице data_crm в колонке APPLICATION_ID составляет:', len(data_crm['APPLICATION_ID'])- len(data_crm['APPLICATION_ID']. drop_duplicates()))
print('Количество дублей в таблице data_crm в колонке INTEGRATION_ID составляет:', len(data_crm['INTEGRATION_ID'])- len(data_crm['INTEGRATION_ID']. drop_duplicates()))

Количество дублей в таблице data_site в колонке APPLICATION_ID составляет: 0
Количество дублей в таблице data_crm в колонке APPLICATION_ID составляет: 0
Количество дублей в таблице data_crm в колонке INTEGRATION_ID составляет: 0


In [6]:
#заливаем в базу
with sqlite3.connect('db') as conn:
    data_site.to_sql(name='site', con=conn, index=False, if_exists='replace')
    data_crm.to_sql(name='crm', con=conn, index=False, if_exists='replace')


Задание 1. Напишите SQL-запрос, который бы возвращал таблицу, на основе которой можно построить воронку продаж от количества заявок на сайте до выдачи с группировкой по продукту. Выдвиньте гипотезу о причине расхождений в количестве заявок между системой сайта и crm-системой.

In [7]:
conn = sqlite3.connect('db')

In [8]:
print('Количество заявок на сайте:', data_site.shape[0])
print('Количество заявок в crm:', data_crm.shape[0])

Количество заявок на сайте: 131149
Количество заявок в crm: 129837


<div style="border:solid steelblue 1px; padding: 20px">
Разница в количестве заявок между системой сайта и crm-системой может быть вызвана:<br />
    * Технический сбой;<br />
    * Бизнес правила (например: к заполнению заявки допускаются только совершенно летние, а заявку заполняли лица младше 18) не пропускают прохождение заявки далее в CRM.<br />

In [9]:
pd.read_sql("""
SELECT ASK_PROD_TYPE_NM as product,
count(INTEGRATION_ID) as sites_count,
count(CLIENT_ID) as crm_count,
sum(APPROVED_FLG) as approved,
sum(ISSUED_FLG) as issued
FROM crm
GROUP BY product
""", conn)

,product,sites_count,crm_count,approved,issued
0,Кредит,63731,63731,8597,1350
1,Кредитная карта,66106,66106,11900,4210


Задание 2
Клиенты могут совершать повторные заявки. Характеристики клиентов, которые делают повторные заявки и которые их не делают, отличаются. Необходимо отсегментировать заявки из таблицы CRM исходя из того, сколько всего клиент сделал заявок. Напишите SQL-запрос, который бы рассчитывал дополнительное поле для таблицы CRM, где находился бы сегмент клиента по признаку количества сделанных заявок (шаг для сегментации задайте произвольно). Рассмотрите воронку продаж в разрезе полученных сегментов. Какие закономерности это позволило выявить? Предложите объяснение.

In [10]:
data_crm.groupby(['CLIENT_ID'])['APPLICATION_ID'].count().reset_index()['APPLICATION_ID'].value_counts()

1     91248
2     13152
3      2421
4       646
5       222
6        78
7        44
8        18
9        13
10        6
13        6
11        4
15        3
12        3
14        2
Name: APPLICATION_ID, dtype: int64

<div style="border:solid steelblue 1px; padding: 20px">
Из группировки, сделанной выше, видно, что:<br />
   * болше всего клиентов с 1-й заявкой<br />
   * количество клиентов с 6 заявками значительно меньше, чем с 5-ю<br />
   * количество людей с 10 и более заявками мало    <br />

In [11]:
pd.read_sql('''


WITH T1 as(
SELECT
CLIENT_ID,
case 
when count(CLIENT_ID)=1 then '1'
when count(CLIENT_ID)>1 and count(CLIENT_ID)<=5 then '1-5'
when count(CLIENT_ID)>5 and count(CLIENT_ID)<=10 then '5-10'
when count(CLIENT_ID)>10 then '>10'
END as segment
FROM crm
GROUP BY CLIENT_ID
)

SELECT ASK_PROD_TYPE_NM as product,
segment,
count(INTEGRATION_ID) as sites_count,
count(crm.CLIENT_ID) as crm_count,
sum(crm.APPROVED_FLG) as approved,
CAST(sum(crm.APPROVED_FLG) AS decimal)/CAST(count(crm.CLIENT_ID) as float)*100 as '%req/approve',
sum(crm.ISSUED_FLG) as issued,
CAST(sum(crm.ISSUED_FLG) as float)/CAST(sum(crm.APPROVED_FLG) AS decimal)*100 as '%approve/issued',
CAST(sum(crm.ISSUED_FLG) AS decimal)/CAST(count(crm.CLIENT_ID) as float)*100 as '%req/issued'
FROM crm
join T1 on crm.CLIENT_ID=T1.CLIENT_ID
GROUP BY product, segment


''', conn)

,product,segment,sites_count,crm_count,approved,%req/approve,issued,%approve/issued,%req/issued
0,Кредит,1,46905,46905,7036,15.000533,1125,15.989198,2.398465
1,Кредит,1-5,16201,16201,1527,9.425344,219,14.341847,1.351768
2,Кредит,5-10,509,509,34,6.679764,6,17.647059,1.178782
3,Кредит,>10,116,116,0,0.000000,0,NaN,0.000000
4,Кредитная карта,1,44343,44343,8883,20.032474,3282,36.946977,7.401394
5,Кредитная карта,1-5,21060,21060,2984,14.169041,911,30.529491,4.325736
6,Кредитная карта,5-10,588,588,31,5.272109,15,48.387097,2.551020
7,Кредитная карта,>10,115,115,2,1.739130,2,100.000000,1.739130


<div style="border:solid steelblue 1px; padding: 20px">
Следует отметить, что с увеличением числа заявок проценты подтверждения и получения кредитных продуктов падает

Задание 3
Напишите sql-запрос, который бы исходя из данных таблицы CRM рассчитывал бы таблицу, содержащую три поля, где id клиента являлось бы уникальным значением, а также признаки «клиент делал заявку на кредит» и «клиент делал заявку на кредитную карту» (1 – делал, неважно сколько раз, 0 – не делал).

<div style="border:solid steelblue 1px; padding: 20px">
Т.к. Outer join не поддерживается, то было реализовано решение через union

In [12]:
pd.read_sql('''

With t1 as (
    SELECT 
        CLIENT_ID as client_id_1, 
        count(INTEGRATION_ID) cnt_1
    FROM 
        crm
    WHERE 
        ASK_PROD_TYPE_NM = 'Кредит'
    GROUP BY 
        CLIENT_ID, ASK_PROD_TYPE_NM
),

t2 as (
    SELECT 
        CLIENT_ID as client_id_2, 
        count(INTEGRATION_ID) cnt_2
    FROM 
        crm
    WHERE 
        ASK_PROD_TYPE_NM = 'Кредитная карта'
    GROUP BY 
        CLIENT_ID, ASK_PROD_TYPE_NM
),

full_join as (
    Select 
        t1.*, t2.* 
    from 
        t1
    left join 
        t2
    on t1.client_id_1 = t2.client_id_2

    UNION ALL 

    Select 
        t1.*, t2.* 
    from 
        t2
    left join 
        t1
    on t1.client_id_1 = t2.client_id_2

    Where t1.cnt_1 is null
)

select
    case 
        when client_id_1 is null 
            then client_id_2
        else client_id_1
        end as client_id,
    case when cnt_1 is null then 0 else 1 end cnt_1,
    case when cnt_2 is null then 0 else 1 end cnt_2
    
from full_join
''', conn)

,client_id,cnt_1,cnt_2
0,1,1,0
1,3,1,0
2,4,1,0
3,6,1,0
4,7,1,0
...,...,...,...
107861,110059,0,1
107862,110060,0,1
107863,110061,0,1
107864,110062,0,1


Задание 4
Напишите SQL-запрос, который исходя из данных таблицы CRM возвращал бы предыдущую заявку клиента на этот же продукт для текущей заявки (например, если текущая заявка – на кредитную карту, то в поле должно содержаться id предыдущей заявки на кредитную карту, а не на кредит). Результат должен содержать четыре поля: APPLICATION_ID, CLIENT_ID, ASK_PROD_TYPE_NM, INTEGRATION_ID, PREV_APPLICATION_ID (предыдущая заявка)

In [13]:
pd.read_sql('''

select 
APPLICATION_ID,
CLIENT_ID,
ASK_PROD_TYPE_NM,
INTEGRATION_ID,
lag(INTEGRATION_ID) over (partition by CLIENT_ID, ASK_PROD_TYPE_NM order by ASK_PROD_TYPE_NM) as PREV_APPLICATION_ID
from crm
order by CLIENT_ID
''', conn)

,APPLICATION_ID,CLIENT_ID,ASK_PROD_TYPE_NM,INTEGRATION_ID,PREV_APPLICATION_ID
0,36169,1,Кредит,SITE-36871,None
1,97949,2,Кредитная карта,SITE-99735,None
2,67786,3,Кредит,SITE-68987,None
3,45381,4,Кредит,SITE-46304,None
4,133092,5,Кредитная карта,SITE-135549,None
...,...,...,...,...,...
129832,37098,110061,Кредитная карта,SITE-37872,None
129833,2759,110062,Кредитная карта,SITE-2765,None
129834,38571,110063,Кредит,SITE-39255,None
129835,112778,110064,Кредитная карта,SITE-114896,None


<div style="border:solid steelblue 1px; padding: 20px">
Мной было выдвинуто предположение, что заявки в crm расположены по порядку их подачи.<br />

In [14]:
data_crm[data_crm['CLIENT_ID'] == 110058]

,APPLICATION_ID,CLIENT_ID,INTEGRATION_ID,ASK_PROD_TYPE_NM,APPROVED_FLG,ISSUED_FLG
381,396,110058,SITE-419,Кредит,1,0
14878,15472,110058,SITE-15760,Кредит,0,0
20969,22087,110058,SITE-22510,Кредит,0,0
104030,107286,110058,SITE-109295,Кредитная карта,0,0
104124,107381,110058,SITE-109416,Кредит,0,0
113666,117116,110058,SITE-119288,Кредитная карта,0,0
